In [3]:
import os
import torch
import torchvision.models as models

# Buat direktori jika belum ada
model_dir = "data/models/"
os.makedirs(model_dir, exist_ok=True)

# Cek dan download ResNet50 jika belum ada
resnet_path = os.path.join(model_dir, "resnet50.pth")
if not os.path.exists(resnet_path):
    resnet = models.resnet50(weights="IMAGENET1K_V1")
    torch.save(resnet.state_dict(), resnet_path)
    print(f"ResNet50 saved to {resnet_path}")
else:
    print(f"ResNet50 already exists at {resnet_path}, skipping download.")

# Cek dan download MobileNetV2 jika belum ada
mobilenet_path = os.path.join(model_dir, "mobilenet_v2.pth")
if not os.path.exists(mobilenet_path):
    mobilenet = models.mobilenet_v2(weights="IMAGENET1K_V1")
    torch.save(mobilenet.state_dict(), mobilenet_path)
    print(f"MobileNetV2 saved to {mobilenet_path}")
else:
    print(f"MobileNetV2 already exists at {mobilenet_path}, skipping download.")


ResNet50 saved to data/models/resnet50.pth
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 50.0MB/s]


MobileNetV2 saved to data/models/mobilenet_v2.pth


# Fungsi Load dan Save Bobot Model (PyTorch/NumPy)

### Fungsi untuk PyTorch (.pth atau .pt)

In [4]:
import torch

def load_model_weights(file_path: str) -> dict:
    """
    Memuat state_dict (bobot) dari file PyTorch (.pth atau .pt).

    Args:
        file_path: Path menuju file bobot model.

    Returns:
        dict: State dictionary (nama lapisan -> tensor bobot).
    """
    try:
        # Gunakan torch.load untuk memuat state_dict.
        # map_location='cpu' disarankan untuk memastikan kompatibilitas antar lingkungan.
        state_dict = torch.load(file_path, map_location='cpu')

        # Asumsi: File .pth berisi state_dict saja.
        # Jika file berisi model lengkap, Anda mungkin perlu memuat arsitektur terlebih dahulu.
        return state_dict
    except Exception as e:
        print(f"Error memuat file PyTorch {file_path}: {e}")
        return {}

def save_model_weights(state_dict: dict, file_path: str):
    """
    Menyimpan state_dict bobot (setelah dimanipulasi) ke file PyTorch.

    Args:
        state_dict: Dictionary bobot yang telah dimodifikasi (hasil dari inject_lsb).
        file_path: Path tujuan untuk menyimpan file bobot baru.
    """
    try:
        torch.save(state_dict, file_path)
        print(f"Bobot model berhasil disimpan ke {file_path}")
    except Exception as e:
        print(f"Error menyimpan file PyTorch ke {file_path}: {e}")

###  Fungsi Alternatif untuk NumPy (.npy atau .npz)

In [5]:
import numpy as np

def load_numpy_array(file_path: str) -> np.ndarray:
    """
    Memuat array bobot tunggal dari file NumPy (.npy).

    Args:
        file_path: Path menuju file .npy.

    Returns:
        np.ndarray: Array bobot yang dimuat.
    """
    try:
        weights = np.load(file_path)
        return weights
    except Exception as e:
        print(f"Error memuat file NumPy {file_path}: {e}")
        return np.array([])

def save_numpy_array(weights_array: np.ndarray, file_path: str):
    """
    Menyimpan array bobot (setelah dimanipulasi) ke file NumPy.

    Args:
        weights_array: Array bobot yang telah dimodifikasi.
        file_path: Path tujuan untuk menyimpan file bobot baru (.npy).
    """
    try:
        np.save(file_path, weights_array)
        print(f"Array bobot berhasil disimpan ke {file_path}")
    except Exception as e:
        print(f"Error menyimpan file NumPy ke {file_path}: {e}")

# LSB Inject

In [6]:
def inject_lsb(model_weights: dict, payload_size_bits: int, target_bit: int) -> dict:
    """
    Menyuntikkan payload biner acak ke posisi bit LSB yang ditentukan
    (misalnya, target_bit=0 untuk LSB pertama mantissa).
    """
    stego_weights = {}
    payload = np.random.randint(0, 2, payload_size_bits) # Payload biner acak
    payload_index = 0

    for name, tensor in model_weights.items():
        if tensor.dtype != torch.float32: # Pastikan hanya bobot float32 yang dimodifikasi
            stego_weights[name] = tensor
            continue

        # Konversi tensor float32 ke representasi biner (NumPy diperlukan)
        weights_flat = tensor.cpu().numpy().flatten()
        weights_view = weights_flat.view(np.int32)

        # Masking dan penyuntikan LSB
        for i in range(len(weights_view)):
            if payload_index < payload_size_bits:
                # Dapatkan nilai integer 32-bit dari float
                int_val = weights_view[i]

                # Buat mask untuk menargetkan bit ke-target_bit (misal: 1 << target_bit)
                mask = 1 << target_bit

                # Hapus bit lama dan sisipkan bit payload baru
                # Posisi LSB untuk mantissa float32 adalah 0 sampai 22

                new_bit = payload[payload_index]

                # Hapus bit lama:
                int_val &= ~mask

                # Sisipkan bit baru:
                int_val |= (new_bit << target_bit)

                weights_view[i] = int_val
                payload_index += 1
            else:
                break

        # Konversi kembali ke tensor dan simpan
        weights_flat_new = weights_view.view(np.float32)
        stego_weights[name] = torch.from_numpy(weights_flat_new.reshape(tensor.shape)).to(tensor.device)

        if payload_index >= payload_size_bits:
            break

    return stego_weights, payload_index


# Generation Datasets

### *Load Models*

In [8]:

weights_cover = load_model_weights(f"{model_dir}/resnet50.pth")

### *LSB Inject*

In [9]:
def decimal_to_short(n):
    """
    Mengkonversi angka desimal ke format singkat, misal:
    1000 -> '1k', 1000000 -> '1M', 1500 -> '1.5k'
    """
    if n >= 1_000_000_000:
        return f"{n/1_000_000_000:.1f}B".rstrip('0').rstrip('.')
    elif n >= 1_000_000:
        return f"{n/1_000_000:.1f}M".rstrip('0').rstrip('.')
    elif n >= 1_000:
        return f"{n/1_000:.1f}k".rstrip('0').rstrip('.')
    else:
        return str(n)


In [18]:
targetBit = 0
payloadSizeBits=500000
weights_stego, bits_injected = inject_lsb(weights_cover, payload_size_bits=payloadSizeBits, target_bit=targetBit)

### *Save Stego Models*

In [19]:
save_model_weights(weights_stego, f"{model_dir}/ResNet50_stego_{targetBit}_{decimal_to_short(payloadSizeBits)}.pth")

Bobot model berhasil disimpan ke data/models//ResNet50_stego_0_500.0k.pth


In [ ]:
print(weights_cover)

In [ ]:
print(weights_stego)

# Feature Extraction

In [20]:
import torch.nn as nn
from scipy.stats import entropy
from sklearn.metrics import mean_squared_error

#### *1. Ekstraksi Loss & Gradient (Memerlukan Model DL)*

In [21]:
def extract_loss_gradient(model: nn.Module, ae_model: nn.Module, data_input: torch.Tensor) -> tuple:
    """
    Menghitung Reconstruction Loss (Skalar) dan Gradient (Vektor/Skalar)
    """
    model.eval()

    # 1. Fitur Reconstruction Loss
    weight_tensor_flat = torch.cat([w.flatten() for w in model.parameters()])
    reconstructed_weights = ae_model(weight_tensor_flat)
    loss_reconstruction = mean_squared_error(weight_tensor_flat.detach().cpu().numpy(),
                                            reconstructed_weights.detach().cpu().numpy())

    # 2. Fitur Gradient
    # Asumsi: Menggunakan CrossEntropyLoss pada satu sampel input
    target = torch.tensor([0]) # Placeholder target class
    criterion = nn.CrossEntropyLoss()

    # Hitung loss dan backpropagate untuk mendapatkan gradien
    output = model(data_input)
    loss_grad = criterion(output, target)
    loss_grad.backward()

    # Ekstraksi Gradien (ambil norm L2 sebagai fitur skalar, atau flatten sebagai vektor)
    gradients = [w.grad.flatten() for w in model.parameters() if w.grad is not None]
    gradient_vector = torch.cat(gradients).detach().cpu().numpy()

    # Menggunakan L2 norm sebagai fitur skalar untuk simplifikasi
    gradient_norm = np.linalg.norm(gradient_vector)

    return loss_reconstruction, gradient_norm # Atau kembalikan gradient_vector jika PSO bisa menangani dimensi besar

#### *2. Ekstraksi Entropi Bit-plane (Memerlukan NumPy)*

In [22]:
def extract_bitplane_entropy(model_weights: dict) -> np.ndarray:
    """
    Menghitung Entropi Shannon untuk 23 bit-plane mantissa (float32)
    """
    entropy_vector = np.zeros(23)

    # Kumpulkan semua bobot float32 ke dalam satu array besar
    all_float_weights = np.concatenate([t.cpu().numpy().flatten()
                                        for t in model_weights.values()
                                        if t.dtype == torch.float32])

    # Konversi ke representasi integer 32-bit
    weights_int_view = all_float_weights.view(np.int32)

    # Analisis Bit-Plane (Mantissa adalah bit ke-0 hingga ke-22)
    for j in range(23):
        # Ekstraksi bit-plane j: (int_value >> j) & 1
        bit_plane = (weights_int_view >> j) & 1

        # Hitung probabilitas p0 dan p1
        p1 = np.mean(bit_plane)
        p0 = 1.0 - p1

        # Hitung Entropi Shannon H = - (p0*log2(p0) + p1*log2(p1))
        # Gunakan np.clip untuk menghindari log(0)
        h = shannon_entropy([p0, p1], base=2)
        entropy_vector[j] = h

    return entropy_vector

#### *3. Penggabungan Fitur*

In [24]:
#Load weights cover models
weights_cover_model = load_model_weights(f"{model_dir}/resnet50.pth")

In [25]:
#Load weights stego models
weights_stego_model = load_model_weights(f"{model_dir}/ResNet50_stego_0_100.0k.pth")

In [ ]:
# F_combined = [loss_reconstruction] + [gradient_norm] + F_Ent
# F_combined adalah Vektor Fitur Berdimensi Tinggi yang menjadi input untuk PSO

#### *Train Autoencoder*

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim

class WeightAutoencoder(nn.Module):
    def __init__(self, input_dim):
        super(WeightAutoencoder, self).__init__()
        # Encoder: Mengurangi dimensi ke hidden_dim (ruang laten)
        hidden_dim = input_dim // 100 # Reduksi drastis untuk kompresi
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, input_dim // 2),
            nn.ReLU(),
            nn.Linear(input_dim // 2, hidden_dim),
            nn.ReLU()
        )
        # Decoder: Merekonstruksi dimensi kembali ke input_dim
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, input_dim // 2),
            nn.ReLU(),
            nn.Linear(input_dim // 2, input_dim),
            nn.Sigmoid() # Atau ReLU, tergantung rentang bobot
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [27]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

def train_autoencoder(cover_weights_list: list, epochs: int = 50, batch_size: int = 32) -> WeightAutoencoder:
    """
    Melatih Autoencoder hanya pada bobot model 'Cover' untuk pembelajaran normal.

    Args:
        cover_weights_list: List dari vektor bobot yang sudah diratakan dari semua model Cover.
        epochs: Jumlah iterasi pelatihan.
        batch_size: Ukuran batch.

    Returns:
        WeightAutoencoder: Model Autoencoder yang telah dilatih.
    """
    if not cover_weights_list:
        raise ValueError("Daftar bobot model Cover tidak boleh kosong.")

    # 1. Persiapan Data
    # Menggabungkan semua bobot Cover menjadi satu tensor matriks
    weights_tensor = torch.stack([torch.tensor(w, dtype=torch.float32) for w in cover_weights_list])
    input_dim = weights_tensor.shape[1]

    # Split data untuk validasi
    X_train, X_val = train_test_split(weights_tensor, test_size=0.1, random_state=42)

    train_dataset = TensorDataset(X_train, X_train) # Input = Output
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # 2. Inisialisasi Model dan Optimizer
    ae_model = WeightAutoencoder(input_dim)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(ae_model.parameters(), lr=0.001)

    # 3. Loop Pelatihan
    ae_model.train()
    print(f"Memulai pelatihan Autoencoder. Dimensi input: {input_dim}")

    for epoch in range(epochs):
        epoch_loss = 0
        for data in train_loader:
            inputs, _ = data

            optimizer.zero_grad()
            outputs = ae_model(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * inputs.size(0)

        avg_loss = epoch_loss / len(train_dataset)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")

        # Tambahkan logika validasi (X_val) di sini jika diperlukan

    # 4. Simpan Model Terlatih
    torch.save(ae_model.state_dict(), "trained_autoencoder_weights.pth")
    return ae_model

In [28]:
# Cek lapisan (layer) yang ada pada model weights_cover_model
if isinstance(weights_cover_model, dict):
    print("Daftar layer pada model (dict keys):")
    for layer_name in weights_cover_model.keys():
        print(layer_name)
elif hasattr(weights_cover_model, 'state_dict'):
    print("Daftar layer pada model (state_dict keys):")
    for layer_name in weights_cover_model.state_dict().keys():
        print(layer_name)
else:
    print("Tipe weights_cover_model tidak dikenali. Tipe:", type(weights_cover_model))

Daftar layer pada model (dict keys):
conv1.weight
bn1.weight
bn1.bias
bn1.running_mean
bn1.running_var
bn1.num_batches_tracked
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.bn1.running_mean
layer1.0.bn1.running_var
layer1.0.bn1.num_batches_tracked
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.bn2.running_mean
layer1.0.bn2.running_var
layer1.0.bn2.num_batches_tracked
layer1.0.conv3.weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.bn3.running_mean
layer1.0.bn3.running_var
layer1.0.bn3.num_batches_tracked
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.0.downsample.1.running_mean
layer1.0.downsample.1.running_var
layer1.0.downsample.1.num_batches_tracked
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.bn1.running_mean
layer1.1.bn1.running_var
layer1.1.bn1.num_batches_tracked
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.bn2.running_mean
layer1.1.bn2.ru

#### *Fungsi Flatten dan Ekstraksi Bobot Cover*

In [29]:
import torch
import numpy as np

# Asumsi fungsi load_model_weights sudah didefinisikan sebelumnya
# def load_model_weights(file_path: str) -> dict: ...

def get_flattened_cover_weights(cover_file_paths: list, target_layer: str = None) -> list:
    """
    Memuat bobot dari daftar file model 'Cover', mengekstrak bobot float32 yang relevan,
    dan meratakannya menjadi vektor tunggal.

    Args:
        cover_file_paths (list): Daftar path string ke file bobot model Cover (misalnya, .pth).
        target_layer (str, optional): Nama lapisan bobot spesifik yang akan diekstrak
                                    (misalnya, 'fc.weight'). Jika None, semua bobot float32 digabungkan.

    Returns:
        list: Daftar vektor NumPy 1D dari bobot model Cover yang diratakan.
    """
    flattened_weights_list = []

    # Menentukan target lapisan bobot mana yang akan diekstrak.
    # Disarankan memilih semua bobot float32, bukan hanya satu lapisan,
    # agar AE belajar representasi global bobot model.

    # 1. Loop Melalui Setiap File Model Cover
    for file_path in cover_file_paths:
        try:
            state_dict = load_model_weights(file_path)
        except Exception as e:
            print(f"Melewati {file_path}: Gagal memuat bobot. {e}")
            continue

        current_model_weights = []

        # 2. Iterasi Melalui Lapisan Bobot dalam state_dict
        for name, tensor in state_dict.items():

            # Hanya proses tensor float32
            if tensor.dtype == torch.float32:

                # Filter berdasarkan lapisan target jika ditentukan
                if target_layer and name != target_layer:
                    continue

                # Pindahkan ke CPU dan konversi ke NumPy array, lalu ratakan (flatten)
                weight_vector = tensor.cpu().numpy().flatten()
                current_model_weights.append(weight_vector)

        if not current_model_weights:
            print(f"Peringatan: Tidak ada bobot float32 yang ditemukan atau lapisan target '{target_layer}' tidak ada di {file_path}.")
            continue

        # 3. Gabungkan Semua Vektor Bobot yang Diekstrak dari SATU MODEL
        # Semua bobot model ResNet50/MobileNetV3 harus memiliki dimensi yang sama
        # (sehingga NumPy.concatenate dapat bekerja)
        try:
            full_flattened_vector = np.concatenate(current_model_weights)
            flattened_weights_list.append(full_flattened_vector)
        except ValueError as e:
            print(f"Kesalahan dimensi pada {file_path}. Pastikan semua model memiliki arsitektur yang sama. {e}")

    # 4. Validasi Dimensi
    if flattened_weights_list:
        dim = flattened_weights_list[0].size
        print(f"\nEkstraksi Selesai: Total {len(flattened_weights_list)} sampel bobot Cover.")
        print(f"Dimensi setiap vektor (Input AE) adalah: {dim} fitur.")

    return flattened_weights_list

In [30]:
# Asumsi Anda sudah mengumpulkan semua path file model Cover Anda
cover_files = [
    f"{model_dir}/resnet50.pth",
    f"{model_dir}/resnet50.pth",
    f"{model_dir}/resnet50.pth",
    # ... Tambahkan sekitar 50-100 sampel model Cover yang berbeda
]

# 1. Dapatkan daftar vektor bobot yang diratakan
cover_weights_list = get_flattened_cover_weights(cover_files)

# # 2. Latih Autoencoder menggunakan daftar vektor bobot ini
# if cover_weights_list:
#     ae_model = train_autoencoder(cover_weights_list, epochs=100)

    # Model ae_model ini sekarang siap digunakan di fungsi extract_loss_gradient!


Ekstraksi Selesai: Total 3 sampel bobot Cover.
Dimensi setiap vektor (Input AE) adalah: 25610152 fitur.


In [31]:
print(cover_weights_list)

[array([ 0.01333477,  0.0146636 , -0.015351  , ..., -0.01300753,
        0.00777029,  0.00243688], dtype=float32), array([ 0.01333477,  0.0146636 , -0.015351  , ..., -0.01300753,
        0.00777029,  0.00243688], dtype=float32), array([ 0.01333477,  0.0146636 , -0.015351  , ..., -0.01300753,
        0.00777029,  0.00243688], dtype=float32)]


In [33]:
ae_model = train_autoencoder(cover_weights_list, epochs=30)

RuntimeError: [enforce fail at alloc_cpu.cpp:124] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 1311759770926208 bytes. Error code 12 (Cannot allocate memory)